In [15]:
from selenium.common import NoSuchElementException
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
import time
import chromedriver_autoinstaller

chromedriver_autoinstaller.install()

options = Options()
options.add_argument("--lang=en_GB")

driver = webdriver.Chrome(options=options)
driver.set_window_size(1024, 600)
driver.maximize_window()

driver.get("https://www.google.com/maps/place/ISCG+Paris/@48.8474575,2.3751911,17z/data=!4m8!3m7!1s0x47e6720facd7c2df:0x4b77c52fd3c34b33!8m2!3d48.8474576!4d2.380062!9m1!1b1!16s%2Fg%2F1tsblfp8?entry=ttu")


names = []
rating = []
dates = []
comments = []

try:
    # Wait for consent page to load
    accept_all_button = WebDriverWait(driver, 10).until(
        # By css_selector (1 or more classes) is more robust choice because by class_name can only handle 1 class
        EC.presence_of_element_located((By.CSS_SELECTOR, "[jsname='b3VHJd']"))
    )
    # Accept cookies because they are good
    accept_all_button.click()
except:
    print("An error occurred when trying to accept Google's terms of conditions.")

# get to reviews
try:
    reviews_button = WebDriverWait(driver, 10).until(
        # By css_selector (1 or more classes) is more robust choice because by class_name can only handle 1 class
        EC.presence_of_element_located((By.CSS_SELECTOR, "[aria-label^='Reviews for']"))
    )
    reviews_button.click()
except:
    print("An error occurred when trying to get to reviews.")

# scroll until the end
try:
    scrollable_div = WebDriverWait(driver, 10).until(
        # By css_selector (1 or more classes) is more robust choice because by class_name can only handle 1 class
        EC.presence_of_element_located((By.CSS_SELECTOR, ".m6QErb.DxyBCb.kA9KIf.dS8AEf"))
    )
    last_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_div)
    print(last_height)
    while True:
        # scroll to the bottom
        driver.execute_script(
            "arguments[0].scrollTop = arguments[0].scrollHeight", scrollable_div
        )
        # wait for the page to load
        time.sleep(2)

        # calculate new scroll height and compare with the last scroll height
        new_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_div)
        print("newheight", new_height, "lastheight", last_height)
        if new_height == last_height:
            break
        last_height = new_height
except:
    print("Couldn't scroll...")

# open the comments fully (see more)
try:
    # gather all see more buttons and click them
    all_see_more_buttons = driver.find_elements(By.CSS_SELECTOR, "[aria-label = 'See more'")
    for button in all_see_more_buttons:
        button.click()
except:
    print("Couldn't click all see more buttons.")

# extract text
try:
    comment_divs = driver.find_elements(By.CSS_SELECTOR, ".jftiEf.fontBodyMedium ")
    # there is necessarily a name, rating and a date but not comment
    for comment in comment_divs:
        names.append(comment.find_element(By.CLASS_NAME, "d4r55 ").text)
        rating.append(comment.find_element(By.CLASS_NAME, "kvMYJc").get_attribute("aria-label"))
        dates.append(comment.find_element(By.CLASS_NAME, "rsqaWe").text)
        # verify that there is comment and add, else add empty string
        text = ""
        try:
            myened_element = comment.find_element(By.CLASS_NAME, "MyEned")
            text = myened_element.find_element(By.CLASS_NAME, "wiI7pd").text
        except NoSuchElementException:
            text = ""
        comments.append(text)
except:
    print("Something happened when trying to parse reviews...")

finally:
    driver.quit()

zipped = list(zip(names, rating, dates, comments))
dictionary = {key: (v1, v2, v3) for key, v1, v2, v3 in zipped}

print(dictionary)

5253
newheight 30342 lastheight 5253
newheight 30342 lastheight 30342
{'Sarah12': ('5 stars', 'a year ago', 'Best school ever'), 'Aïchata Traoré': ('5 stars', '4 months ago', "Le personnel de l'école s'investit dans la satisfaction du parcours en formation de ses candidats ainsi que de la réussite aux examens. L'ensemble de l'équipe pédagogique s'assure du bien-être des élèves au sein de leur formation et entreprise.\nLes étudiants ont la possibilité d'emprunter du matériel informatique en cas de nécessité.\nJe souligne également que l'accompagnement dans la recherche d'entreprise est très satisfaisant."), 'Inès Delafolie': ('5 stars', '4 months ago', "Une excellente école ! Un corps enseignant à l'écoute des élèves, des intervenants qui ont cœur à transmettre ce qu'ils connaissent. Un très bon accompagnement dans l'ensemble, surtout lorsque je cherchais une entreprise d'accueil !\nToutes les conditions sont réunies pour s'épanouir dans la formation choisie, vraiment top et j'espère po